In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.callbacks as callbacks
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import json


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
print('tensorflow version', tf.version.VERSION)

1 Physical GPUs, 1 Logical GPUs
tensorflow version 2.3.2


In [2]:
def load_data():
  headline = []
  label = []
  with open('sarcasm/sarcasm.json', 'r') as f:
    raw = json.load(f)
    for entry in raw:
      headline.append(entry['headline'])
      label.append(entry['is_sarcastic'])
  max_length = max([len(h) for h in headline])
  print('max_length: ', max_length)
  assert len(headline) == len(label)
  max_classes = max([l for l in label])
  print('class: ', max_classes)
  return headline, label


VOCAB_SIZE = 6000
MAX_LENGTH = 46

data, label = load_data()
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<UNKNOWN_TOKEN>',
                                                  num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(data)
print('data size: ', len(data))
# print(tokenizer.word_index)

sequences = tokenizer.texts_to_sequences(data)
tokens = keras.preprocessing.sequence.pad_sequences(sequences,
                                                    padding='post',
                                                    truncating='post',
                                                    maxlen=MAX_LENGTH)
assert len(tokens) == len(label)
print(data[0])
print('->')
print(tokens[0])
print('token: ', tokens.shape)

max_length:  254
class:  1
data size:  26709
former versace store clerk sues over secret 'black code' for minority shoppers
->
[ 308    1  679 3337 2298   48  382 2576    1    6 2577    1    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
token:  (26709, 46)


In [3]:
BATCH_SIZE = 128

training_size = int(len(data) * 0.8)
validation_size = len(data) - training_size
ds = tf.data.Dataset.from_tensor_slices((tokens, label))
training_ds = ds.take(training_size).batch(BATCH_SIZE)
validation_ds = ds.skip(training_size).batch(BATCH_SIZE)


for example in training_ds.take(1):
  example_data, example_label = example
  print(example_data[0])
  print(example_label[0])

tf.Tensor(
[ 308    1  679 3337 2298   48  382 2576    1    6 2577    1    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0], shape=(46,), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)


In [6]:
EMBEDDING_SIZE = 16


def embedding(x):
  position = tf.range(start=0, limit=MAX_LENGTH, delta=1)
  em = layers.Embedding(VOCAB_SIZE, EMBEDDING_SIZE, mask_zero=True)(x)
  pos_em = layers.Embedding(MAX_LENGTH, EMBEDDING_SIZE)(position)
  return em + pos_em


def transformer(x, dropout=0.2):
  m = tfa.layers.MultiHeadAttention(EMBEDDING_SIZE, 8)([x, x, x])
  d = layers.Dropout(dropout)(m)
  a = layers.Add()([x, d])
  x = layers.LayerNormalization(epsilon=1e-6)(a)
  x = layers.Dense(EMBEDDING_SIZE, activation='relu')(x)
  x = layers.LayerNormalization(epsilon=1e-6)(x)
  x = layers.Dropout(dropout)(x)
  return x


def build_model():
  inputs = keras.Input(shape=[MAX_LENGTH], dtype=tf.int32)
  em = embedding(inputs)
  x = transformer(em)
  x = layers.GlobalAveragePooling1D()(x)
  x = layers.Dense(20, activation="relu")(x)
  outputs = layers.Dense(2, activation="softmax")(x)
  model = keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer=keras.optimizers.Adam(1e-4),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


keras.backend.clear_session()
model = build_model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 46)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 46, 16)       96000       input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_AddV2 (TensorFlowOp [(None, 46, 16)]     0           embedding[0][0]                  
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 46, 16)       8208        tf_op_layer_AddV2[0][0]          
                                                                 tf_op_layer_AddV2[0][0

In [7]:
tensorboard_callback = callbacks.TensorBoard()
checkpoint_callback = callbacks.ModelCheckpoint('sarcasm-model/sarcasm.{epoch:02d}-{val_loss:.2f}.h5',
                                                monitor='val_accuracy',
                                                verbose=1,
                                                save_best_only=True)


history = model.fit(training_ds, epochs=10, validation_data=validation_ds,
                    callbacks=[tensorboard_callback, checkpoint_callback])

Epoch 1/10
165/167 [============================>.] - ETA: 0s - loss: 0.6986 - accuracy: 0.5324
Epoch 00001: val_accuracy improved from -inf to 0.56870, saving model to sarcasm-model/sarcasm.01-0.67.h5
167/167 [==============================] - 1s 8ms/step - loss: 0.6983 - accuracy: 0.5327 - val_loss: 0.6702 - val_accuracy: 0.5687
Epoch 2/10
162/167 [============================>.] - ETA: 0s - loss: 0.6607 - accuracy: 0.6005
Epoch 00002: val_accuracy improved from 0.56870 to 0.66455, saving model to sarcasm-model/sarcasm.02-0.64.h5
167/167 [==============================] - 1s 8ms/step - loss: 0.6602 - accuracy: 0.6020 - val_loss: 0.6437 - val_accuracy: 0.6645
Epoch 3/10
167/167 [==============================] - ETA: 0s - loss: 0.6171 - accuracy: 0.7113
Epoch 00003: val_accuracy improved from 0.66455 to 0.78061, saving model to sarcasm-model/sarcasm.03-0.58.h5
167/167 [==============================] - 1s 8ms/step - loss: 0.6171 - accuracy: 0.7113 - val_loss: 0.5832 - val_accuracy: 0.

In [8]:
import tensorflowjs as tfjs

model.save('sarcasm.h5')
tfjs.converters.save_keras_model(model, 'sarcasm.tfjs')

/home/kiddos/.local/lib/python3.6/site-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
